In [10]:
from PIL import Image
import json

def construct_mcq(options, correct_option):
    correct_option_letter = None
    i = "a"
    mcq = ""

    for option in options:
        if option == correct_option:
            correct_option_letter = i
        mcq += f"{i}. {option}\n"
        i = chr(ord(i) + 1)

    if correct_option_letter is None:
        print(options, correct_option)
        raise ValueError("Correct option not found in the options")
    
    mcq = mcq[:-1]
    return mcq, correct_option_letter

def resize_image(image_path, size):
    '''resize image so that the largest edge is atmost size'''
    img = Image.open(image_path)
    width, height = img.size

    if width <= size and height <= size:
        return img
    
    if width > height:
        new_width = size
        new_height = int(height * (size / width))
    else:
        new_height = size
        new_width = int(width * (size / height))
    img = img.resize((new_width, new_height), Image.Resampling.LANCZOS)
    return img


def add_row(content, data, i, with_answer=False):  

    content.append("Image "+str(i)+": ")
    content.append(resize_image(data["image_path"], 512))
    content.append(data["question"])
    content.append(data["mcq"])

    if with_answer:
        content.append("Answer {}: {}".format(i, data["correct_option_letter"]))
    else:
        content.append("Answer {}: ".format(i))
    
    return content
   

In [11]:
import google.generativeai as genai

GOOGLE_API_KEY='YOUR_API_KEY'

FEWSHOT_JSON = "illusionVQA/comprehension/fewshot_labels.json"
FEWSHOT_IMAGE_DIR = "illusionVQA/comprehension/FEW_SHOTS/"
EVAL_JSON = "illusionVQA/comprehension/eval_labels.json"
EVAL_IMAGE_DIR = "illusionVQA/comprehension/EVAL/"

genai.configure(api_key=GOOGLE_API_KEY)
model_name = 'gemini-1.5-flash-002'
model = genai.GenerativeModel(model_name)

In [12]:
with open(FEWSHOT_JSON) as f:
    fewshot_dataset = json.load(f)

for data in fewshot_dataset:
    data["image_path"] = FEWSHOT_IMAGE_DIR + data["image"]
    data["mcq"], data["correct_option_letter"] = construct_mcq(data["options"], data["answer"])

In [13]:

import json
with open(EVAL_JSON) as f:
    eval_dataset = json.load(f)

from collections import defaultdict

category_count = defaultdict(int)
import os
for data in eval_dataset:
    if data["image"] not in os.listdir(EVAL_IMAGE_DIR):
        print(data["image"])
        continue
    data["image_path"] = EVAL_IMAGE_DIR + data["image"]
    data["mcq"], data["correct_option_letter"] = construct_mcq(data["options"], data["answer"])
    category_count[data["category"]] += 1

print(category_count)
print(len(eval_dataset))

defaultdict(<class 'int'>, {'color': 23, 'real-scene': 64, 'impossible object': 134, 'angle illusion': 26, 'circle-spiral': 6, 'hidden': 45, 'perspective': 2, 'angle constancy': 2, 'deceptive design': 37, 'edited-scene': 21, 'occlusion': 2, 'size': 46, 'positive-negative space': 7, 'counting': 11, 'repeating pattern': 2, 'upside-down': 7})
435


### 0-shot

In [14]:

content = ["You'll be given an image, an instruction and some options. You have to select the correct one. Do not explain your reasoning. Answer with the option's letter from the given choices directly."
]

next_data_idx = 1

### 4-shot

In [15]:
# content = ["You'll be given an image, an instruction and some choices. You have to select the correct one. Do not explain your reasoning. Answer with the option's letter from the given choices directly. Here are a few examples:"
# ]

# i = 1
# for data in fewshot_dataset:
#     content = add_row(content, data, i, with_answer=True)
#     i += 1
# content.append("Now you try it.")

# next_data_idx = i

In [16]:
from tqdm import tqdm
ytrue = []
ypred = []

In [17]:
from google.generativeai.types import HarmCategory, HarmBlockThreshold

for  data in tqdm(eval_dataset):
    content_t = add_row(content.copy(), data, next_data_idx, with_answer=False)
    # print(content_t)
    while True:
        try:
            response = model.generate_content(content_t,
                                              safety_settings={
                HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_NONE,
                HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_NONE,
                HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_NONE,
                HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_NONE
            }

            )
            break
        except Exception as e:
            print(e)
            print("Internal Error")
            continue
    
    try:
        gemini_answer = response.text
    except Exception as e:
        try:
            gemini_answer = response.parts[0].text
        except Exception as e:
            print("External Error:", response.prompt_feedback)

            gemini_answer = str(response.prompt_feedback)
        
    if gemini_answer[-1] == ".":
        gemini_answer = gemini_answer[:-1]
    gemini_answer = gemini_answer[0].lower()

    answer = data["correct_option_letter"]
    data[model_name+"_answer"] = gemini_answer

100%|██████████| 435/435 [03:47<00:00,  1.91it/s]


In [19]:


with open(model_name+"_results.json", "w") as f:
    json.dump(eval_dataset, f)
